In [1]:
import pandas as pd
import os
import pyarrow.parquet as pq

In [2]:
#Mortalidade infantil até 5 anos
directory = '../Necessidades Humanas Básicas/obitos/base'
file_list = [f for f in os.listdir(directory) if f.endswith('.parquet')]

In [9]:
dfs = []
for file_name in file_list[-2:]:
    file_path = os.path.join(directory, file_name)
    table = pq.read_table(file_path)

    df = table.to_pandas()
    dfs.append(df)

df = pd.concat(dfs)

df = df[["ano", "sigla_uf", "id_municipio_ocorrencia","idade", "linha_a",  "linha_b", "linha_c", "linha_d", "linha_ii","obito_gravidez", "obito_puerperio", "circunstancia_obito"]]

In [4]:
df

,ano,sigla_uf,id_municipio_ocorrencia,idade,linha_a,linha_b,linha_c,linha_d,linha_ii,obito_gravidez,obito_puerperio,circunstancia_obito
0,2020,PA,1501402,72.0,*A419,*A418,*L021,None,None,None,None,None
1,2020,GO,5208707,50.0,*T811,*J958,*K632*T818,*Y838,*E669,None,None,None
2,2020,PA,5107909,71.0,*A419,*K650,*K819,*B342*U071,*I10X*E149,None,None,None
3,2020,MA,2111300,0.0,*P285,*P220,*P072,*P072*P027,None,None,None,None
4,2020,PE,2616407,69.0,*J81X,*I509,*I110,*I10X,*J449,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
778384,2020,DF,5300108,42.0,*A419,*N390*B961,*M329,None,NaN,NaN,NaN,None
778385,2020,RN,2408102,38.0,*A419,*J189,*F842,None,NaN,NaN,NaN,None
778386,2020,RN,2408102,35.0,*J81X,*I219,*I259*I500,*F172,NaN,NaN,NaN,None
778387,2020,RN,2408102,84.0,*I619,None,None,None,NaN,NaN,NaN,None


In [10]:
def remove_x(line):

    if line.endswith("X"):
        line = line[:-1]

    return line
def clean_lines(
        obitos,
        lines=[
            "linha_a",
            "linha_b",
            "linha_c",
            "linha_d",
            "linha_ii"]):

    for line in lines:
        obitos[line] = obitos[line].fillna("Sem causa").apply(
            lambda x: x.split("*"))

    obitos.reset_index(
        drop=True, inplace=True)
    obitos = obitos.reset_index().rename(
        columns={"index":"id"})

    for line in lines:
        obitos = obitos.explode([line])
        obitos = obitos[
            obitos[line].str.strip() != '']

    dfs = []
    for line in lines:
        df = obitos[
            ['id',
             'ano',
             'sigla_uf',
             'id_municipio_ocorrencia',
             'idade',
             f"{line}",
             "obito_gravidez",
             "circunstancia_obito"]]

        df = df.rename(
            columns={f"{line}":"causa", "circunstancia_obito":"circunstancia"})
        dfs.append(df)

    obitos = pd.concat(dfs)

    obitos = obitos.drop_duplicates()
    obitos = obitos[obitos["causa"] != "Sem causa"]

    obitos["causa"] = obitos["causa"].apply(lambda x: remove_x(x))

    obitos.sort_values(by=["id", "causa"], inplace=True)
    obitos.reset_index(drop=True, inplace=True)

    return obitos

In [11]:
obitos = clean_lines(df)

In [17]:
obitos.to_parquet("C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/bases_tratadas/obitos_tratados_microdados.parquet")

In [34]:
df = df[["ano", "sigla_uf", "id_municipio_ocorrencia","idade"]].query("idade<=5")
df = df.groupby(["ano", 	"sigla_uf",	"id_municipio_ocorrencia"], as_index=False).count()

,ano,sigla_uf,id_municipio_ocorrencia,idade
0,2020,AC,1100205,1
1,2020,AC,1200054,1
2,2020,AC,1200104,4
3,2020,AC,1200203,39
4,2020,AC,1200302,16
...,...,...,...,...
2439,2020,TO,5103403,1
2440,2020,TO,5204904,1
2441,2020,TO,5208707,5
2442,2020,TO,5220207,1


In [25]:
df[["ano", "sigla_uf", "id_municipio_ocorrencia","idade",  'obito_gravidez']]

,ano,sigla_uf,id_municipio_ocorrencia,idade,obito_gravidez
0,2020,GO,5217401,42.0,None
1,2020,SC,4209102,81.0,None
2,2020,GO,5202502,91.0,None
3,2020,SC,4204202,47.0,None
4,2020,PA,1505908,89.0,None
...,...,...,...,...,...
778384,2020,DF,5300108,42.0,2
778385,2020,RN,2408102,38.0,2
778386,2020,RN,2408102,35.0,2
778387,2020,RN,2408102,84.0,None
